# **Monk 1**

### Dataset

In [ ]:
import pandas as pd
from io import StringIO
import numpy as np
from utils import get_splits
from sklearn.preprocessing import MinMaxScaler

def retrieveData(data_str):
    # Create a DataFrame from the structured data
    column_names = ["R", "col2", "col3", "col4", "col5", "col6", "col7", "data"]
    data = pd.read_csv(StringIO(data_str), sep=' ', header=None, names=column_names)
    data = data.iloc[:, :-1]
    data=data.iloc[np.random.permutation(len(data))]
    scaler = MinMaxScaler()

    df_scaled = scaler.fit_transform(data.to_numpy())
    df_scaled = pd.DataFrame(df_scaled, columns=data.columns.values)
    del df_scaled['R']
    #del df_scaled['Id']
    df_scaled = df_scaled.assign(R=data['R'].values)
    df_train = df_scaled
    features = 6
    X_train = df_train.iloc[ : , :features].values
    y_train = df_train.iloc[:,features:].values
    return X_train, y_train

filename    = "monk/monks-1"
train       = ".train"
test        = ".test"

f = open(filename + train, 'r')
res = f.readlines()
f.close()
train_str = ''.join(res)
X_train, y_train = retrieveData(train_str)

f = open(filename + test, 'r')
res = f.readlines()
f.close()
test_str = ''.join(res)
X_test, y_test = retrieveData(test_str)

### Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from activation_function import Tanh, Linear, ReLU, Sigmoid
from layer import Layer
from mlp import MLP
from metrics import Metrics
from losses import MeanSquaredError
from grid_search import GridSearch

In [ ]:
print(y_train)

In [ ]:
t = GridSearch()
t.read_json(["models/model1.json", "models/model2.json", "models/model3.json"])
t.set_dataset(X_train, y_train)
t.test_grid()

In [ ]:
layers = [
    Layer(
        n_perceptrons=8,
        n_inputs=len(X_train[0]),
        act_func=ReLU(),
        #kernel_regularizer=0.0001,
        #bias_regularizer=0.01,
        momentum=0.9,
        Nesterov=True
    ),
    Layer(
        n_perceptrons=len(y_train[0]),
        n_inputs=8,
        act_func=Sigmoid(),
        #kernel_regularizer=0.0001,
        #bias_regularizer=0.01,
        momentum=0.9,
        Nesterov=True
    )
]

model = MLP(layers)

epochs, lr = 100, 0.03
loss = MeanSquaredError()

model.compile(lr,loss, metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=epochs)
model.summary()
model.plot_error()

In [ ]:
track = Metrics()
error = 0  
for i in range(len(X_train)):
    out = model.run(X_train[i])
    print(f'case {i}: {round(out[0], 2)}, {y_train[i][0]}')
    track.compute__results(round(out[0]), y_train[i][0])
print("accuracy: ", track.accuracy())

In [ ]:
track = Metrics()
error = 0  
for i in range(len(X_test)):
    out = model.run(X_test[i])
    print(f'case {i}: {round(out[0], 2)}, {y_test[i][0]}')
    track.compute__results(round(out[0]), y_test[i][0])
print("accuracy: ", track.accuracy())